In [1]:
import argparse
import datetime
import math
import os
import sys
import random
import pickle
import numpy as np
import pandas as pd
from PIL import Image
from omegaconf import OmegaConf
sys.path.append('/data2/youngju/kimst24')

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from torchvision import datasets, models, transforms
from tqdm.auto import tqdm
from utils.load_dataset import get_CUB_loaders

/home/youngju/anaconda3/envs/proto/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("--data-path", type=str, default="/data/CUB_200_2011/images")
parser.add_argument("--model-path", type=str, default="/data2/youngju/kimst24/outputs/clf_checkpoints/2024-04-27_132424/clf_ep-11_lr-0.001_val-acc-0.8609.pth")
parser.add_argument("--pickle-path", type=str, default="/data2/youngju/kimst24/outputs/get_config/test")
parser.add_argument("--phase", type=str, default="test")
parser.add_argument("--num-pickle", type=int, default=7)
parser.add_argument("--seed", type=int, default=0)
parser.add_argument("--batch-size", type=int, default=512)
parser.add_argument("--device", type=str, default="cuda:5")
parser.add_argument("--train-ratio", type=float, default=0.7)

args = parser.parse_args(args=[])

In [3]:
MEAN = np.array([0.485, 0.456, 0.406])
STD = np.array([0.229, 0.224, 0.225])

random.seed(args.seed)
torch.manual_seed(args.seed)
cudnn.deterministic = True
cudnn.benchmark = False

(train_loader, train_data_len) = get_CUB_loaders(args.data_path, args.batch_size, args.train_ratio, train=True)
(val_loader, test_loader, valid_data_len, test_data_len) = get_CUB_loaders(args.data_path, int(args.batch_size/2), args.train_ratio, train=False)

In [4]:
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(2048, 200)
model.load_state_dict(torch.load(args.model_path))
model = model.to(args.device)
model.eval()

/home/youngju/anaconda3/envs/proto/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/youngju/anaconda3/envs/proto/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
## activation
ac_list = []
for i in tqdm(range(args.num_pickle)): # pickle 파일 개수
    with open(f"{args.pickle_path}/{args.phase}_activation_{i}.pkl","rb") as fr:
        ac_batch = pickle.load(fr)
    ac_list.append(ac_batch[-1])
ac_total = torch.cat(ac_list, dim=0)
with open(f"{args.pickle_path}/total_activation.pkl", "wb") as fw:
    pickle.dump(ac_total, fw)

100%|██████████| 7/7 [00:28<00:00,  4.04s/it]


In [6]:
### eq
eq_list = []
for i in tqdm(range(args.num_pickle)): # pickle 파일 개수
    with open(f"{args.pickle_path}/{args.phase}_eq_{i}.pkl","rb") as fr:
        eq = pickle.load(fr)
    eq_list.append(eq)
eq_total = torch.cat(eq_list, dim=0)
with open(f"{args.pickle_path}/total_eq.pkl", "wb") as fw:
    pickle.dump(eq_total, fw)

100%|██████████| 7/7 [00:00<00:00, 3782.06it/s]


In [7]:
### x
x_list = []
for i in tqdm(range(args.num_pickle)): # pickle 파일 개수
    with open(f"{args.pickle_path}/{args.phase}_x_{i}.pkl","rb") as fr:
        x = pickle.load(fr)
    x_list.append(x)
x_total = torch.cat(x_list, dim=0)
with open(f"{args.pickle_path}/total_x.pkl", "wb") as fw:
    pickle.dump(x_total, fw)

100%|██████████| 7/7 [00:00<00:00, 11.79it/s]


In [8]:
### y
y_list = []
for i in tqdm(range(args.num_pickle)): # pickle 파일 개수
    with open(f"{args.pickle_path}/{args.phase}_y_{i}.pkl","rb") as fr:
        y = pickle.load(fr)
    y_list.append(y)
y_total = torch.cat(y_list, dim=0)
with open(f"{args.pickle_path}/total_y.pkl", "wb") as fw:
    pickle.dump(y_total, fw)

100%|██████████| 7/7 [00:00<00:00, 3723.54it/s]
